### Exploritory Analysis Notebook 

This notebook will outline the analysis we took for this dataset

##### Read in the curated data

##### Handling Any missing data

##### Descriptive statistics for them 

##### Distributions of features and Outlier analysis 

##### Scaling?? 

##### Pairwise Plots 

##### Feature Relations + Combination PCA

##### Most expensive suburbs (mapped? by rent / wealth affluence etc)